In [1]:
import pandas as pd
from decimal import Decimal
from re import sub
from nltk.corpus import stopwords
import string
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

from nltk.stem import PorterStemmer

# Yelp data

In [2]:
dfb = pd.read_pickle('data/YelpBusiness.pkl.gz')
dfr = pd.read_pickle('data/YelpReviewStemmed.pkl.gz')

In [3]:
dfb.head(2)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state,relevance
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON,1
2,"10110 Johnston Rd, Ste 15","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...",gnKjwL_1w79qoiV3IC_xQQ,"Sushi Bars, Restaurants, Japanese",Charlotte,"{'Monday': '17:30-21:30', 'Wednesday': '17:30-...",1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC,1


In [4]:
def check_toronto(city):
    if isinstance(city, str):
        if city == 'Toronto':
            return 1
        else:
            return 0
    else:
        return 0

In [5]:
dfb['in_toronto'] = dfb['city'].progress_apply(check_toronto)

In [6]:
den = len(dfb)
dfb = dfb[dfb.in_toronto == 1]
b_ids = set(dfb.business_id)
enum = len(dfb)
print('{:.1f}% of relevant Yelp businesses are in Toronto'.format( enum/den*100 ) ) 

13.5% of relevant Yelp businesses are in Toronto


In [7]:
den = len(dfr)
dfr = dfr[dfr.business_id.isin(b_ids)]
enum = len(dfr)
print('{:.1f}% of relevant Yelp reviews are in Toronto'.format( enum/den*100 ) ) 

9.2% of relevant Yelp reviews are in Toronto


In [8]:
print(len(dfr))
dfr = dfr.dropna(subset=['stemmed_text'])
print(len(dfr))

436095
436091


In [9]:
dfb.to_pickle('data/TorontoBusiness.pkl.gz')

In [10]:
dfr.to_pickle('data/TorontoReview.pkl.gz')

# Airbnb data

In [11]:
dfal = pd.read_csv('data/TorontoAirbnbListings.csv.gz', compression='gzip')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
dfal['price_float'] = dfal.apply(lambda x: float(Decimal(sub(r'[^\d.]', '', x.price))), axis=1)

In [13]:
stop_words = set(stopwords.words('english'))
punct = list(string.punctuation)
ps = PorterStemmer()

def clean_text(text):
    if isinstance(text, str):
        for ch in punct:
            text = text.replace(ch, '')
        word_tokens = text.lower().split() 
        clean_text = [ps.stem(w) for w in word_tokens if not w in stop_words]
        ct = ' '.join(clean_text)
        return ct
    else:
        return None
dfal['stemmed_neighborhood_overview'] = dfal['neighborhood_overview'].progress_apply(clean_text)

In [14]:
dfal.to_pickle('data/TorontoAirbnbListings.pkl.gz')